In [ ]:
import numpy.random as npr
import time

from jax import jit,grad
from jax.scipy.special import logsumexp
import jax.numpy as jnp
import datasets

In [ ]:
def init_random_params(scale,layer_sizes,rng=npr.RandomState(0)):
  return [(scale*rng.randn(m,n),scale*rng.randn(n)) for m,n in zip(layer_sizes[:-1],layer_sizes[1:])]

In [ ]:
param_scale=0.1
layer_sizes=[784,1024,1024,10]
step_size=0.001
num_epochs=10
batch_size=128

In [ ]:
def predict(params,inputs):
  activations=inputs
  for w,b in params[:1]:
    outputs=jnp.dot(activations,w)+b
    activations=jnp.tanh(outputs)

  final_w,final_b=params[-1]
  logits=jnp.dot(activations,final_w)+final_b
  return logits-logsumexp(logits,axis=1,keepdims=True)    

In [ ]:
def loss(params,batch):
  inputs,targets=batch
  preds=predict(params,inputs)
  return -jnp.mean(jnp.sum(preds*targets,axis=1))

In [ ]:
def accuracy(params,batch):
  inputs,targets=batch
  target_class=jnp.argmax(targets,axis=1)
  predicted_class=jnp.argmax(predict(params,inputs),axis=1)
  return jnp.mean(predicted_class==target_class)

In [ ]:
train_images,train_labels,test_images,test_labels=datasets.mnist()
num_train=train_images.shape[0]
num_complete_batches,leftover=divmod(num_train,batch_size)
num_batches=num_complete_batches+bool(leftover)

downloaded https://storage.googleapis.com/cvdf-datasets/mnist/train-images-idx3-ubyte.gz to /tmp/jax_example_data/
downloaded https://storage.googleapis.com/cvdf-datasets/mnist/train-labels-idx1-ubyte.gz to /tmp/jax_example_data/
downloaded https://storage.googleapis.com/cvdf-datasets/mnist/t10k-images-idx3-ubyte.gz to /tmp/jax_example_data/
downloaded https://storage.googleapis.com/cvdf-datasets/mnist/t10k-labels-idx1-ubyte.gz to /tmp/jax_example_data/


In [ ]:
print(num_batches)

469

In [ ]:
def data_stream():
  rng=npr.RandomState(0)
  while True:
    perm=rng.permutation(num_train)
    for i in range(num_batches):
      batch_idx=perm[i*batch_size:(i+1)*batch_size]
      yield train_images[batch_idx],train_labels[batch_idx]  

batches=data_stream()

In [ ]:
@jit
def update(params,batch):
  grads=grad(loss)(params,batch)
  return [(w-step_size*dw,b-step_size*db) for (w,b),(dw,db) in zip(params,grads)]

In [ ]:
params=init_random_params(param_scale,layer_sizes)

In [ ]:
print(len(params))

3


In [ ]:
print(params[0][0].shape)

(784, 1024)


In [ ]:
print(params[0][1].shape)

(1024,)


In [ ]:
for epoch in range(num_epochs):
  start_time=time.time()
  for _ in range(num_batches):
    params=update(params,next(batches))
  epoch_time=time.time()-start_time

train_acc=accuracy(params,(train_images,train_labels))
test_acc=accuracy(params,(test_images,test_labels))
print("Epoch {} in {:0.2f} sec".format(epoch,epoch_time))
print("Training set accuracy {}".format(train_acc))
print("Test set accuracy {}".format(test_acc))

Epoch 9 in 0.39 sec
Training set accuracy 0.898900032043457
Test set accuracy 0.9048000574111938
